# **🥈This Notebook now rank 15 in LB!**
+ I used `TruncatedSVD(n_components=128, random_state=42)` both on train and test data.
+ CatBoostRegressor is used. To save time， only trained on fold_0. Maybe more fold will help to impore score.
+ This notebook is based on [FABIEN CROM](https://www.kaggle.com/code/fabiencrom/msci-multiome-quickstart-w-sparse-matrices). Please Upvoted if help !
+ Version_1 is a quick submission and version_2_3 show the training process.

In [ ]:
!pip install colorama
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 110 kB/s 


In [ ]:
import os, gc, pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from colorama import Fore, Back, Style
from matplotlib.ticker import MaxNLocator

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, scale
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

import scipy
import scipy.sparse

import gc
import pickle
import warnings
warnings.filterwarnings('ignore')

## CFG & import & scoring func

### CFG

In [ ]:
#CONFIG

CFG_seed = 42
CFG_n_folds = 5
CFG_target = 'target'
CFG_DEBUG = True
CFG_SUBMIT = False
CFG_RETRAIN = False
CFG_is_Colab = True

In [ ]:
import os

if CFG_is_Colab:
    FP_INPUT_DIR = '/content/drive/MyDrive/kaggle/single-cell-competition/data/input/'
    FP_OUTPUT_DIR = '/content/drive/MyDrive/kaggle/single-cell-competition/data/output/'
    
    from google.colab import drive
    drive.mount('/content/drive')
else:
    FP_INPUT_DIR = '../input/'
    FP_OUTPUT_DIR = './'


FP_SPARSE_DATA = os.path.join(FP_INPUT_DIR, "multimodal-single-cell-as-sparse-matrix/")

FP_CELL_METADATA = os.path.join(FP_SPARSE_DATA,"metadata.parquet")

FP_CITE_TRAIN_INPUTS_VALUES = os.path.join(FP_SPARSE_DATA,"train_cite_inputs_values.sparse.npz")
FP_CITE_TRAIN_INPUTS_IDXCOL = os.path.join(FP_SPARSE_DATA,"train_cite_inputs_idxcol.npz")
FP_CITE_TRAIN_TARGETS_VALUES = os.path.join(FP_SPARSE_DATA,"train_cite_targets_values.sparse.npz")
FP_CITE_TRAIN_TARGETS_IDXCOL = os.path.join(FP_SPARSE_DATA,"train_cite_targets_idxcol.npz")
FP_CITE_TEST_INPUTS_VALUES = os.path.join(FP_SPARSE_DATA,"test_cite_inputs_values.sparse.npz")
FP_CITE_TEST_INPUTS_IDXCOL = os.path.join(FP_SPARSE_DATA,"test_cite_inputs_idxcol.npz")

FP_MULTI_TRAIN_INPUTS_VALUES = os.path.join(FP_SPARSE_DATA,"train_multi_inputs_values.sparse.npz")
FP_MULTI_TRAIN_INPUTS_IDXCOL = os.path.join(FP_SPARSE_DATA,"train_multi_inputs_idxcol.npz")
FP_MULTI_TRAIN_TARGETS_VALUES = os.path.join(FP_SPARSE_DATA,"train_multi_targets_values.sparse.npz")
FP_MULTI_TRAIN_TARGETS_IDXCOL = os.path.join(FP_SPARSE_DATA,"train_multi_targets_idxcol.npz")
FP_MULTI_TEST_INPUTS_VALUES = os.path.join(FP_SPARSE_DATA,"test_multi_inputs_values.sparse.npz")
FP_MULTI_TEST_INPUTS_IDXCOL = os.path.join(FP_SPARSE_DATA,"test_multi_inputs_idxcol.npz")

FP_SUBMISSION = os.path.join(FP_SPARSE_DATA,"sample_submission.parquet")
FP_EVALUATION = os.path.join(FP_SPARSE_DATA,"evaluation.parquet")

Mounted at /content/drive


### import

In [ ]:
%pip install colorama
%pip install seaborn

import os, gc, pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from colorama import Fore, Back, Style
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import os,gc,time,random
from tqdm.notebook import tqdm

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, scale
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

import scipy
import scipy.sparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### The scoring function (from AmbrosM)

This competition has a special metric: For every row, it computes the Pearson correlation between y_true and y_pred, and then all these correlation coefficients are averaged.

In [ ]:
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    if y_true.shape != y_pred.shape: raise ValueError("Shapes are different.")
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)


# Preprocessing and cross-validation

We first load all of the training input data for Multiome. It should take less than a minute.

In [ ]:
%%time
train_inputs = scipy.sparse.load_npz(FP_MULTI_TRAIN_INPUTS_VALUES)

CPU times: user 42.2 s, sys: 5.77 s, total: 48 s
Wall time: 1min 6s


In [ ]:
train_inputs = train_inputs.astype('float16', copy=False)
gc.collect()

103

## PCA / TruncatedSVD
It is not possible to directly apply PCA to a sparse matrix, because PCA has to first "center" the data, which destroys the sparsity. This is why we apply `TruncatedSVD` instead (which is pretty much "PCA without centering"). It might be better to normalize the data a bit more here, but we will keep it simple.

In [ ]:
%%time
n_components = 2
pca = TruncatedSVD(n_components=n_components, random_state=42)
train_inputs = pca.fit_transform(train_inputs)
print(pca.explained_variance_ratio_.sum())

0.004117675
CPU times: user 2min 12s, sys: 10.2 s, total: 2min 22s
Wall time: 2min 24s


In [ ]:
%%time
train_targets = scipy.sparse.load_npz(FP_MULTI_TRAIN_TARGETS_VALUES)

CPU times: user 19.6 s, sys: 1.77 s, total: 21.3 s
Wall time: 23.5 s


In [ ]:
%%time
n_components2 = 2
pca2 = TruncatedSVD(n_components=n_components2, random_state=42)
train_targets_pca = pca2.fit_transform(train_targets)
print(pca2.explained_variance_ratio_.sum())

0.043512993
CPU times: user 39.8 s, sys: 1.25 s, total: 41.1 s
Wall time: 42 s


In [ ]:
def save(name, model):
    with open(FP_OUTPUT_DIR + name, 'wb') as f:
        pickle.dump(model, f)
        
def load_pkl(name):
    with open(name, 'rb') as pickle_file:
        return pickle.load(pickle_file)

In [ ]:
n_components = 2
n_components2 = 2

save(f'pca_{n_components}comp.pkl', pca)
save(f'pca2_{n_components}comp.pkl', pca2)
save(f'train_multi_inputs_svd_2.pkl', train_inputs)
save(f'train_multi_targets_svd_2.pkl', train_targets_pca)

In [ ]:
!pip install catboost

from catboost import CatBoostRegressor
params = {'learning_rate': 0.1, 
          'depth': 7, 
          'l2_leaf_reg': 4, 
          'loss_function': 'MultiRMSE', 
          'eval_metric': 'MultiRMSE', 
          'task_type': 'CPU', 
          'iterations': 200,
          'od_type': 'Iter', 
          'boosting_type': 'Plain', 
          'bootstrap_type': 'Bayesian', 
          'allow_const_label': True, 
          'random_state': 1
         }
model = CatBoostRegressor(**params)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.4 MB/s 


In [ ]:
train_inputs = load_pkl(FP_OUTPUT_DIR + 'train_multi_inputs_svd_2.pkl')
train_targets_pca = load_pkl(FP_OUTPUT_DIR + 'train_multi_targets_svd_2.pkl')
pca = load_pkl(FP_OUTPUT_DIR + 'pca_2comp.pkl')
pca2 = load_pkl(FP_OUTPUT_DIR + 'pca2_2comp.pkl')

In [ ]:
n = 1000

In [ ]:
np.random.seed(42)
all_row_indices = np.arange(train_inputs.shape[0])
np.random.shuffle(all_row_indices)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

index = 0
score = []

# model = Ridge(copy_X=False)
d = train_inputs.shape[0]//n
for i in range(0, n*d, d):
    print(f'start [{i}:{i+d}]')
    ind = all_row_indices[i:i+d]    
    for idx_tr, idx_va in kf.split(ind):
        X = train_inputs[ind]
        Y = train_targets_pca[ind] #.todense()
        Yva = train_targets[ind][idx_va]
        Xtr, Xva = X[idx_tr], X[idx_va]
        Ytr = Y[idx_tr]
        del X, Y
        gc.collect()
        print('Train...')
        model.fit(Xtr, Ytr)
        del Xtr, Ytr
        gc.collect()
        s = correlation_score(Yva.todense(), model.predict(Xva)@pca2.components_)
        score.append(s)
        print(index, s)
        del Xva, Yva
        gc.collect()
        pkl_filename =FP_OUTPUT_DIR + f"model{index:02d}.pkl"
        index += 1
        with open(pkl_filename, 'wb') as file:
            pickle.dump(model, file)
    #     break
    # break
    gc.collect()

NameError: ignored

In [ ]:
del train_targets_pca, train_inputs, train_targets
gc.collect()

8


# Predicting

In [ ]:
%%time
multi_test_x = scipy.sparse.load_npz(FP_MULTI_TEST_INPUTS_VALUES)
multi_test_x = pca.transform(multi_test_x)

CPU times: user 25.7 s, sys: 1.22 s, total: 26.9 s
Wall time: 28.7 s


In [ ]:
# test_sd = np.std(multi_test_x, axis=1).reshape(-1, 1)
# test_sd[test_sd == 0] = 1
# test_norm = (multi_test_x - np.mean(multi_test_x, axis=1).reshape(-1, 1)) / test_sd
# test_norm = test_norm.astype(np.float16)
# del multi_test_x
# gc.collect()

In [ ]:
test_len = multi_test_x.shape[0]
d = test_len//n
x = []
for i in range(n):
    x.append(multi_test_x[i*d:i*d+d])
del multi_test_x
gc.collect()

68

In [ ]:
index

1

In [ ]:
preds = np.zeros((test_len, 23418), dtype='float16')
for i,xx in enumerate(x):
    for ind in range(index):
        # print(ind, end=' ')
        with open(FP_OUTPUT_DIR + f'model{ind:02}.pkl', 'rb') as file:
            model = pickle.load(file)
        preds[i*d:i*d+d,:] += (model.predict(xx)@pca2.components_)/index
        gc.collect()
    # print('')
    del xx
gc.collect()

In [ ]:
del x
gc.collect()

114

In [ ]:
np.save(FP_OUTPUT_DIR+ 'preds.npy', preds)

# Creating submission

We load the cells that will have to appear in submission.

In [ ]:
preds = np.load(FP_INPUT_DIR+ 'preds.npy')

In [ ]:
%%time
# Read the table of rows and columns required for submission
eval_ids = pd.read_parquet(FP_EVALUATION)
# Convert the string columns to more efficient categorical types
#eval_ids.cell_id = eval_ids.cell_id.apply(lambda s: int(s, base=16))
eval_ids.cell_id = eval_ids.cell_id.astype(pd.CategoricalDtype())
eval_ids.gene_id = eval_ids.gene_id.astype(pd.CategoricalDtype())

CPU times: user 31.2 s, sys: 5.74 s, total: 36.9 s
Wall time: 35.2 s


In [ ]:
# Prepare an empty series which will be filled with predictions
submission = pd.Series(name='target',
                       index=pd.MultiIndex.from_frame(eval_ids), 
                       dtype=np.float32)
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86              NaN
1         c2150f55becb  CD274             NaN
2         c2150f55becb  CD270             NaN
3         c2150f55becb  CD155             NaN
4         c2150f55becb  CD112             NaN
                                           ..
65744175  2c53aa67933d  ENSG00000134419   NaN
65744176  2c53aa67933d  ENSG00000186862   NaN
65744177  2c53aa67933d  ENSG00000170959   NaN
65744178  2c53aa67933d  ENSG00000107874   NaN
65744179  2c53aa67933d  ENSG00000166012   NaN
Name: target, Length: 65744180, dtype: float32

We load the `index`  and `columns` of the original dataframe, as we need them to make the submission.

In [ ]:
%%time
y_columns = np.load(FP_MULTI_TRAIN_TARGETS_IDXCOL,
                   allow_pickle=True)["columns"]

test_index = np.load(FP_MULTI_TEST_INPUTS_IDXCOL,
                    allow_pickle=True)["index"]

CPU times: user 72.3 ms, sys: 13.1 ms, total: 85.4 ms
Wall time: 847 ms


We assign the predicted values to the correct row in the submission file.

In [ ]:
cell_dict = dict((k,v) for v,k in enumerate(test_index)) 
assert len(cell_dict)  == len(test_index)

gene_dict = dict((k,v) for v,k in enumerate(y_columns))
assert len(gene_dict) == len(y_columns)

In [ ]:
eval_ids_cell_num = eval_ids.cell_id.apply(lambda x:cell_dict.get(x, -1))
eval_ids_gene_num = eval_ids.gene_id.apply(lambda x:gene_dict.get(x, -1))

valid_multi_rows = (eval_ids_gene_num !=-1) & (eval_ids_cell_num!=-1)

In [ ]:
submission.iloc[valid_multi_rows] = preds[eval_ids_cell_num[valid_multi_rows].to_numpy(),
eval_ids_gene_num[valid_multi_rows].to_numpy()]

In [ ]:
del eval_ids_cell_num, eval_ids_gene_num, valid_multi_rows, eval_ids, test_index, y_columns
gc.collect()

In [ ]:
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86                    NaN
1         c2150f55becb  CD274                   NaN
2         c2150f55becb  CD270                   NaN
3         c2150f55becb  CD155                   NaN
4         c2150f55becb  CD112                   NaN
                                             ...   
65744175  2c53aa67933d  ENSG00000134419    5.917969
65744176  2c53aa67933d  ENSG00000186862    0.016953
65744177  2c53aa67933d  ENSG00000170959    0.041656
65744178  2c53aa67933d  ENSG00000107874    1.217773
65744179  2c53aa67933d  ENSG00000166012    4.578125
Name: target, Length: 65744180, dtype: float32

# Merging with CITEseq predictions

We use the CITEseq predictions from [this notebook](https://www.kaggle.com/code/vuonglam/lgbm-baseline-optuna-drop-constant-cite-task) by VuongLam.

In [ ]:
submission.reset_index(drop=True, inplace=True)
submission.index.name = 'row_id'

In [ ]:
cite_submission = pd.read_csv(FP_INPUT_DIR + "msci-citeseq-keras-quickstart/submission.csv")
cite_submission = cite_submission.set_index("row_id")
cite_submission = cite_submission["target"]

In [ ]:
submission[submission.isnull()] = cite_submission[submission.isnull()]

In [ ]:
submission

row_id
0           0.094605
1          -0.162362
2          -0.405332
3          -0.302582
4           1.114355
              ...   
65744175    5.917969
65744176    0.016953
65744177    0.041656
65744178    1.217773
65744179    4.578125
Name: target, Length: 65744180, dtype: float32

In [ ]:
submission.isnull().any()

False

In [ ]:
submission.to_csv(FP_OUTPUT_DIR + "submission.csv")

In [ ]:
!head submission.csv

head: cannot open 'submission.csv' for reading: No such file or directory


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def set_cfg():
    import os
    import json
    f = open("/content/drive/MyDrive/api/kaggle.json", 'r')
    json_data = json.load(f)
    os.environ['KAGGLE_USERNAME'] = json_data['username']
    os.environ['KAGGLE_KEY'] = json_data['key']

def submit():
    !pip install kaggle
    !kaggle competitions submit -c open-problems-multimodal -f /content/drive/MyDrive/kaggle/single-cell-competition/data/output/submission.csv -m "Message"
def check_submissions():
    !kaggle competitions submissions -c open-problems-multimodal


In [ ]:
if CFG_is_Colab:
    set_cfg()
    # submit()
    check_submissions()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
100% 1.19G/1.19G [00:16<00:00, 77.0MB/s]
Successfully submitted to Open Problems - Multimodal Single-Cell IntegrationfileName                                     date                 description                                                       status    publicScore  privateScore  
-------------------------------------------  -------------------  ----------------------------------------------------------------  --------  -----------  ------------  
submission.csv                               2022-11-04 00:45:09  Message                                                           pending                              
submission.csv                               2022-11-03 03:40:12  Message                                                           complete  0.800                      
submission.csv                               2022-11-03 03:30:30  Message                               

fileName                                     date                 description                                                       status    publicScore  privateScore  
-------------------------------------------  -------------------  ----------------------------------------------------------------  --------  -----------  ------------  
submission.csv                               2022-11-03 03:30:30  Message                                                           complete  0.800                      
[🥈LB_T15| MSCI Multiome] CatBoostRegressor   2022-10-25 11:51:14                                                                    complete  0.810                      
submission.csv                               2022-10-21 07:12:02                                                                    complete  0.798                      
MSCI Multiome Quickstart w/ Sparse Matrices  2022-10-15 00:07:20                                                                    complete  0.802   